In [10]:
import os
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub

load_dotenv()
embedding = OpenAIEmbeddings(model = 'text-embedding-3-large'
                             , dimensions=1024) # 벡터 dimension에 맞게 조절가능. default openAI embedding: 3072

# readonly. 조회용. 
index_name = 'law-index'
query = '원자력안전법 1조에 대한 내용을 요약해서 알려주세요'
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# 쿼리, 갯수에 대해 유사검색을 처리
retrieve_documents = database.similarity_search(query, k=3)

llm = ChatOpenAI(model = "gpt-4o-mini")
prompt = hub.pull('rlm/rag-prompt')
retriever = database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever, # Chroma뿐 아니라, 파인콘, AzureAISearch 등 다른곳에서도 사용이 가능함.
    chain_type_kwargs = {"prompt": prompt}
)
print(f'1st prompt: {prompt}')

dictionary = ["원자력안전법 -> 방사선안전법"]
rephrase_prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다. 이 경우엔, 답변만 반환합니다.
    사전: {dictionary}
    질문: {{question}}
""")
print(f'1st rephrase_prompt: {rephrase_prompt}')

# 모델을 통해 답변을 받아 전처리 해주는 코드.
dictionary_chain = rephrase_prompt | llm | StrOutputParser()
# new_question = dictionary_chain.invoke({"question": query}) # -> {{question}}

law_chain = {"query": dictionary_chain} | qa_chain
ai_response = law_chain.invoke({"question": query})
print(f'ai_response: {ai_response}')


1st prompt: input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]
1st rephrase_prompt: input_variables=['question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template="\n    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.\n    만약 변경할 필요가 없다고 판단되